# score_evaluating

In [2]:
import re
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from collections import Counter
from konlpy.tag import Mecab
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tqdm import tqdm
from util import dbhelper

In [335]:
from tensorflow.keras.layers import Embedding, Dense, GRU
from tensorflow.keras.models import Sequential
from tensorflow.keras.models import load_model
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

## 1. '여기어때' 사이트 리뷰 데이터의 긍정/부정 예측하기

In [ ]:
def sentiment_predict(data):
    # 1차 전처리
    data = only_ks(data)
    data = remove_null(data)
    data = no_repeat(data)
    data = remove_whitespace(data)
    data = remove_dupli(data)
    data = spell_check(data)
    data = remove_null(data)
    
    # 2차 전처리
    temp_list = []
    for i in tqdm(range(len(data))):
        temp_dict = {}
        text = mecab.morphs(data['review'][i])
        encoded = tokenizer.texts_to_sequences([text])
        pad_new = pad_sequences(encoded, maxlen = max_len)

        score = float(loaded_model.predict(pad_new))
        temp_dict['tokenized_review'] = text
        if(score > 0.5):
            temp_dict['percent'] = round(score * 100, 2)
            temp_dict['sentiment'] = '긍정'
            
        else:
            temp_dict['percent'] = round((1 - score) * 100, 2)
            temp_dict['sentiment'] = '부정'
        temp_list.append(dict(temp_dict))
    
    temp_df = pd.DataFrame(temp_list)
    new_df = pd.concat([data, temp_df], axis=1)
        
    return new_df

## 2. 정확도 평가

In [6]:
data['score_sentiment'] = np.select([data.score > 4], ['긍정'], default='부정')
data['correct'] = np.select([data.score_sentiment == data.sentiment], ['O'], default='X')
data

,name,review,tokenized_review,score,score_sentiment,percent,sentiment,correct
0,코트야드 메리어트 서울 보타닉 파크,파크뷰 정말 예뻤습니다,"['파크뷰', '예뻤']",5,긍정,85.46,긍정,X
1,오라카이 대학로 호텔,께끗해용,"['께', '끗', '해용']",5,긍정,80.34,긍정,X
2,코코모 호텔,깔끔하고 좋았습니다 또 와도 좋을 것 같아요,"['깔끔', '좋', '와도', '좋']",5,긍정,89.98,긍정,X
3,신라스테이 서초,깨끗하고 매우 좋았어요,"['깨끗', '좋']",5,긍정,88.35,긍정,X
4,목시 바이 메리어트 서울 인사동,번째 이용하는데도 늘 완벽,"['째', '늘', '완벽']",5,긍정,95.19,긍정,X
...,...,...,...,...,...,...,...,...
44711,오라카이 대학로 호텔,항상 잘 머물다 갑니다,"['항상', '머물']",5,긍정,91.74,긍정,X
44712,호텔 크레센도 서울,좁은 거 빼고는 다 좋았습니다,"['좁', '좋']",5,긍정,50.87,부정,X
44713,호텔 드 어반,두어 반의 최강 메리트는 층에 무한 제공되는 시간 셀프 바가 있다는 것 초인 우리...,"['두어', '반', '최강', '메리트', '무한', '제공', '시간', '셀프...",5,긍정,71.44,긍정,X
44714,뜨랑블루 호텔,에어컨 조명 통합 기능 리모컨이 없어서 아쉬웠어요,"['에어컨', '조명', '통합', '기능', '리모컨', '없', '아쉬웠']",5,긍정,92.11,부정,X


In [1]:
O_cnt = 0
X_cnt = 0
for i in data['correct']:
    if i == 'O':
        O_cnt += 1
    else:
        X_cnt += 1

correct = round((O_cnt / len(data)) * 100, 2)
print('O 카운트 :', O_cnt)
print('X 카운트 :', X_cnt)
print('정확도 : {}%'.format(correct))

NameError: name 'data' is not defined